In [2]:
# %%
# import library's
import time
from selenium import webdriver
import csv
import urllib.request as req
import random
from datetime import datetime
import pathlib
from selenium.webdriver.common.by import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import pandas as pd
import os
import json
from urllib.parse import urlparse
import io
from PIL import Image
import logging
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from pathlib import Path
import os
import sys
from googleapiclient.discovery import build
from google.cloud import storage
from PIL import Image
import io


project_init=False
if not os.path.isfile('config.csv'):
    config_df=pd.DataFrame(columns=["SUMRUSH_EMAIL","GOOGLE_CUSTOM_SEARCH_ENGINE_ID","GOOGLE_CUSTOM_SEARCH_API_KEY","GCS_S3_BUCKET_NAME","GCS_CREDENTIALS_PATH"])    
    config_df.to_csv('config.csv',index=False)
    project_init=True
else:
    config_df=pd.read_csv('config.csv')
    
 

if not os.path.isfile('proxies.csv'):
    proxy_df=pd.DataFrame(columns=["smurush_email","proxy"])    
    proxy_df.to_csv('proxies.csv',index=False)
    project_init=True
else:
    config_df=pd.read_csv('proxies.csv')

if project_init:   
    sys.exit('Config And Proxy Just Intilized')
 
with open('config/global_config.json', 'r') as file:
    CONFIG_DATA = json.load(file)

# GOOGLE_CUSTOM_SEARCH_API_KEY=CONFIG_DATA['GOOGLE_CUSTOM_SEARCH_API_KEY']
# GOOGLE_CUSTOM_SEARCH_ENGINE_ID=CONFIG_DATA['GOOGLE_CUSTOM_SEARCH_ENGINE_ID']

# GCS_S3_BUCKET_NAME=CONFIG_DATA['GCS_S3_BUCKET_NAME']
# GCS_CREDENTIALS_PATH=Path.cwd()+'/'+CONFIG_DATA['GCS_CREDENTIALS_PATH']


In [ ]:

def _get_gcs_public_url(bucket_name: str, object_name: str, secure: bool=True):
    scheme = "https" if secure else "http"
    return f"{scheme}://storage.googleapis.com/{bucket_name}/{object_name}"



def _upload_to_gcs(bucket_name: str, file_name: str, buffer: bytes):

    client = storage.Client.from_service_account_json(GCS_CREDENTIALS_PATH)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_string(buffer)

    return _get_gcs_public_url(bucket_name, file_name)


def rtn_url(image_path,website):
    image = Image.open(image_path)

    with io.BytesIO() as buffer:
        image.save(buffer, format="PNG")
        
        return _upload_to_gcs(GCS_S3_BUCKET_NAME, website+str(time.time()).split('.')[0]+'.png', buffer.getvalue())


# Create custom search service
custom_search_service = build(
    "customsearch", "v1", developerKey=GOOGLE_CUSTOM_SEARCH_API_KEY
)


# Get estimated indexed page numbers using the google custom search api
def _get_estimated_indexed_pages(service, api_key, cx, domain):
    query = f"site:{domain}"
    response = service.cse().list(q=query, cx=cx, key=api_key).execute()
    if 'error' in response:
        if response['error']['code'] == 403 and response['error']['errors'][0]['reason'] == 'dailyLimitExceeded':
            raise Exception("Your daily quota has been exhausted.")
        else:
            raise Exception(f"An error occurred: {response['error']['message']}")

    return response['searchInformation']['totalResults']


def get_estimated_pages(domain_with_scheme: str) -> int:
    return _get_estimated_indexed_pages(
        custom_search_service,
        GOOGLE_CUSTOM_SEARCH_API_KEY,
        GOOGLE_CUSTOM_SEARCH_ENGINE_ID,
        domain_with_scheme
    )



# Create custom search service
custom_search_service = build(
    "customsearch", "v1", developerKey=GOOGLE_CUSTOM_SEARCH_API_KEY
)


# Get estimated indexed page numbers using the google custom search api
def _get_estimated_indexed_pages(service, api_key, cx, domain):
    query = f"site:{domain}"
    response = service.cse().list(q=query, cx=cx, key=api_key).execute()
    
    if 'error' in response:
        if response['error']['code'] == 403 and response['error']['errors'][0]['reason'] == 'dailyLimitExceeded':
            raise Exception("Your daily quota has been exhausted.")
        else:
            raise Exception(f"An error occurred: {response['error']['message']}")

    return response['searchInformation']['totalResults']


def get_estimated_pages(domain_with_scheme: str) -> int:
    return _get_estimated_indexed_pages(
        custom_search_service,
        GOOGLE_CUSTOM_SEARCH_API_KEY,
        GOOGLE_CUSTOM_SEARCH_ENGINE_ID,
        domain_with_scheme
    )

# import library's

In [2]:

# login system
def login():
    inp=input('Do you want to Start Login System(Y/N)? ')
    if inp.lower()=='y':
        options = webdriver.ChromeOptions() 
        if CONFIG_DATA['SHOW_Window_on_login'].lower()!='yes':
            options.add_argument('headless')

        driver = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.
        driver.implicitly_wait(10)
        print('Starting.....')

        # %%
        file_path = 'config/smush_account.txt'  # Replace 'your_file.txt' with the actual file path
        try:
            with open(file_path, 'r') as file:
                # Read the entire contents of the file
                file_contents = file.read()
                file_contents=file_contents.split('\n')
                for i in file_contents:
                    driver.get('https://www.semrush.com/')
                    try:
                        driver.find_element(By.CSS_SELECTOR, "#srf-search-bar")
                        print(email, 'Just Logged In a while ago')
                        continue
                    except:
                        pass
                        
                    try:
                        driver.find_element(By.CSS_SELECTOR, ".ch2-dialog-actions  .ch2-allow-all-btn").click()
                    except:
                        pass
                    email=i.split(':')[0]
                    password=i.split(':')[1]


                    if os.path.exists('smush_cookies/'+email+'_cookies.json'):
                        with open('smush_cookies/ijguk787duis@icznn.com_cookies.json', 'r') as file:
                            cookies = json.load(file)
                            for cookie in cookies:
                                driver.add_cookie(cookie)
                            driver.refresh()
                        try:
                            driver.find_element(By.CSS_SELECTOR, "#srf-search-bar")
                            print(email, 'logged in from before')
                            continue
                        except:
                            pass
                        

                    driver.find_element(By.CSS_SELECTOR, "#loginForm #email").send_keys(email)
                    driver.find_element(By.CSS_SELECTOR, "#loginForm #password").send_keys(password)
                    # driver.find_element(By.CSS_SELECTOR, "#loginForm button[type=submit]").click()
                    driver.execute_script("arguments[0].click()", driver.find_element(By.CSS_SELECTOR, "#loginForm button[type=submit]"))
                    time.sleep(10)
                    try:
                        driver.find_element(By.CSS_SELECTOR, 'button[data-test="header-menu__user"]')
                        f=True
                        pass
                    except Exception as e:
                        f=False
                        print(email, 'Is Bad Credentials',e)
                        pass
                    if f==True:
                        cookies = driver.get_cookies()
                        with open('smush_cookies/'+email+'_cookies.json', 'w') as file:
                            json.dump(cookies, file)
                        print(email, 'Is logged in succesfully')
                        driver.delete_all_cookies()
                    
            driver.quit()
        except FileNotFoundError:
            print(f"The file  was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")
    inp=input('Do you want to Login Another Account (Y/N)? ')
    if inp.lower()=='y':
        login()
# login system

In [3]:
# check if file is csv or excel
def is_csv(filename):
    # Get the file extension from the filename
    file_extension = os.path.splitext(filename)[1].lower()

    # Check if the file extension is CSV or Excel
    if file_extension in ['.csv']:
        return True
    else:
        return False
# check if file is csv or excel

In [ ]:
if __name__ == "__main__" or True:
    login()
    # intilize data files
    print('Intializing Data Files.....')
    while True:
        filename = input('Write Your Data File Name(Usa1.xlsx): ')
        file_path='data_files/'+filename
        if os.path.isfile(file_path):
            break
        else:
            print(f"The file {file_path} does not exist. Try Again.")
            continue

    if is_csv(file_path):
        df=pd.read_csv(file_path)
    else:
        df=pd.read_excel(file_path)
    if not "Authority Score" in df.columns:
        df['Authority Score'] = None
    if not "Organic Search Traffic" in df.columns:
        df['Organic Search Traffic'] = None
    if not "ImageURL" in df.columns:
        df['ImageURL'] = None
    if not "Page View" in df.columns:
        df['Page View'] = None
    
    if is_csv(file_path):
        df=df.to_csv(file_path,index=False)
    else:
        df=df.to_excel(file_path,index=False)
    
    
    
    
    # intilize data files